In [2]:
import krakenex
from requests.exceptions import HTTPError
import pprint

In [3]:
kraken = krakenex.API()

In [28]:
def get_order_book(pair, count):
    """Queries the kraken order book.

    Parameters
    ----------
    pair : str 
        Asset pair to get market depth for.
    count : int 
        Maximum number of asks/bids

    Returns
    -------
    dict
        asks : list
            Ask side array of (<price>, <volume>, <timestamp>).
        bids : list 
            Bid side array of (<price>, <volume>, <timestamp>).
    """
    
    try:
        response = kraken.query_public('Depth', {'pair': ['XXBTZUSD', 'XXRPZEUR'], 'count': '5'})
    except HTTPError as e:
        print("ERROR: Failed getting order book data for pair={}".format(pair))
    
    obook = response.get('result', {}).get(pair)
    if not obook:
        print("WARNING: Empty response.")

    return obook

In [29]:
get_order_book('XXRPZEUR', 30)

{'asks': [['0.27860000', '898.744', 1552255394],
  ['0.27861000', '220.690', 1552255293],
  ['0.27869000', '5509.644', 1552255346],
  ['0.27870000', '400.000', 1552255278],
  ['0.27883000', '1300.018', 1552255357]],
 'bids': [['0.27845000', '981.000', 1552255424],
  ['0.27826000', '15000.964', 1552255425],
  ['0.27823000', '25111.650', 1552255428],
  ['0.27814000', '5026.560', 1552255425],
  ['0.27813000', '3594.988', 1552255370]]}